<a href="https://colab.research.google.com/github/engtasneemalassaf-tech/Clinical-Symptom-Screening-Assistant-for-Nurses/blob/main/Clinical_Symptom_Screening_Assistant_(for_Nurses)22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🩺 Medical Screening Assistant  
**Chatbot to Help Nurses in Early Patient Triage**  
  

**Objective:** Build a nurse-friendly prototype covering all rubric items at Distinction level.  
**Models:**  
- Model A: Logistic Regression (sklearn)  
- Model B: Flan-T5-base (fast, CPU-friendly, public)  

**Datasets:**  
- Main: itachi9604/disease-symptom-description-dataset  
- Secondary: uom190346a/disease-symptoms-and-patient-profile-dataset  

**Final UI:** Clean, professional, nurse-focused (symptom input, extracted symptoms, Model A top predictions, Model B guidance, escalation note).

# **1.Install Dependencies**

In [1]:
!pip install -q gradio==4.42.0 pandas scikit-learn fuzzywuzzy python-Levenshtein kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 1.0.1 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
google-adk 1.21.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
google-genai 1.55.0 requires websockets<1

# **2.Download Datasets (kagglehub)**

In [2]:
import kagglehub
import os

print("Downloading main dataset...")
path1 = kagglehub.dataset_download("itachi9604/disease-symptom-description-dataset")
print("Dataset 1 path:", path1)

print("Downloading secondary dataset...")
path2 = kagglehub.dataset_download("uom190346a/disease-symptoms-and-patient-profile-dataset")
print("Dataset 2 path:", path2)

print("Files in main dataset:", os.listdir(path1))
print("Files in secondary dataset:", os.listdir(path2))

100%|██████████| 30.1k/30.1k [00:00<00:00, 24.2MB/s]

Extracting files...
Dataset 1 path: /root/.cache/kagglehub/datasets/itachi9604/disease-symptom-description-dataset/versions/2


100%|██████████| 3.07k/3.07k [00:00<00:00, 5.36MB/s]

Extracting files...
Dataset 2 path: /root/.cache/kagglehub/datasets/uom190346a/disease-symptoms-and-patient-profile-dataset/versions/2
Files in main dataset: ['symptom_Description.csv', 'symptom_precaution.csv', 'dataset.csv', 'Symptom-severity.csv']
Files in secondary dataset: ['Disease_symptom_and_patient_profile_dataset.csv']


# **3.Load Data & Create Lookup Maps**

In [3]:
import pandas as pd

data_file = os.path.join(path1, "dataset.csv")
desc_file = os.path.join(path1, "symptom_Description.csv")
prec_file = os.path.join(path1, "symptom_precaution.csv")

df = pd.read_csv(data_file)
desc_df = pd.read_csv(desc_file)
prec_df = pd.read_csv(prec_file)

desc_map = dict(zip(desc_df["Disease"].str.strip(), desc_df["Description"].str.strip()))

prec_map = {}
for _, row in prec_df.iterrows():
    d = str(row["Disease"]).strip()
    p = [str(row[c]).strip() for c in prec_df.columns if "precaution" in c.lower() and pd.notna(row[c])]
    prec_map[d] = ", ".join(p)

print(f"Loaded {len(df)} records, {len(desc_map)} diseases, {len(prec_map)} precautions")

Loaded 4920 records, 41 diseases, 41 precautions


# **4.Train Model A (Logistic Regression)**

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Print columns of df to debug the KeyError
print("Columns in df:", df.columns.tolist())

target_col = "Disease" # This has been corrected based on actual df columns
X_raw = df.drop(columns=[target_col])
y_raw = df[target_col].astype(str)

symptom_vocab = sorted(set(
    str(s).strip().lower().replace(" ", "_")
    for col in X_raw.columns
    for s in X_raw[col].dropna().astype(str)
    if str(s).strip().lower() not in ["", "nan"]
))
symptom_index = {s: i for i, s in enumerate(symptom_vocab)}

def row_to_multihot(row):
    vec = np.zeros(len(symptom_vocab), dtype=int)
    for val in row:
        if pd.isna(val): continue
        s = str(val).strip().lower().replace(" ", "_")
        if s in symptom_index:
            vec[symptom_index[s]] = 1
    return vec

X = np.array([row_to_multihot(r) for _, r in X_raw.iterrows()])
le = LabelEncoder()
y = le.fit_transform(y_raw)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

lr = LogisticRegression(max_iter=2000, n_jobs=-1)
lr.fit(X_train, y_train)

acc = lr.score(X_test, y_test)
print(f"Model A (LR) test accuracy: {acc:.4f} ({acc*100:.2f}%)")

Columns in df: ['Disease', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17']
Model A (LR) test accuracy: 1.0000 (100.00%)


# **5.Load Model B (Flan-T5-base – CPU friendly)**

In [7]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)
print("Model B loaded (fast on CPU)")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Model B loaded (fast on CPU)


# **6.Core Functions (Extraction, Prompt, Escalation)**

In [11]:
import re
from fuzzywuzzy import process

DISCLAIMER = "This tool is for educational purposes only and should not be used as a substitute for professional medical advice. Always consult with a qualified healthcare provider for any health concerns."

def extract_symptoms(text):
    t = re.sub(r"[^a-z0-9\s]", " ", text.lower())
    t = re.sub(r"\s+", " ", t).strip()
    found = []
    for s in symptom_vocab:
        if s.replace("_", " ") in t:
            found.append(s)
    for w in t.split():
        if len(w) > 4:
            match, score = process.extractOne(w, [s.replace("_", " ") for s in symptom_vocab])
            if score >= 88:
                found.append(match.replace(" ", "_"))
    return sorted(set(found))

def run_ui(user_text):
    symptoms = extract_symptoms(user_text)
    symptoms_str = ", ".join(symptoms) if symptoms else "none"

    if not symptoms:
        return symptoms_str, "(no match)", "Describe symptoms clearly.", "No escalation"

    vec = np.zeros(len(symptom_vocab))
    for s in symptoms:
        if s in symptom_index:
            vec[symptom_index[s]] = 1
    vec = vec.reshape(1, -1)

    probs = lr.predict_proba(vec)[0]
    top_idx = np.argsort(probs)[::-1][:5]
    top_pairs = [(le.inverse_transform([i])[0], float(probs[i])) for i in top_idx]

    a_out = ""
    for rank, (d, p) in enumerate(top_pairs, 1):
        if p < 0.05: continue
        desc = desc_map.get(d, "No desc")[:300] + "..."
        prec = prec_map.get(d, "No prec")[:200] + "..."
        a_out += f"{rank}. {d} (Conf: {p:.1%})\nDesc: {desc}\nPrec: {prec}\n---\n"

    top_d_str = ", ".join([d for d,p in top_pairs[:3]])
    prompt = f"""You are a nurse triage assistant. Symptoms: {symptoms_str}. Reference ML: {top_d_str}. Give safe advice, suggest nurse actions, end with disclaimer."""
    b_out = pipe(prompt, max_length=250)[0]["generated_text"].strip()
    b_out += "\n\n" + DISCLAIMER

    escal = []
    if top_pairs[0][1] < 0.35: escal.append("Low confidence")
    if len(symptoms) < 3: escal.append("Few symptoms")
    if any(w in user_text.lower() for w in ["chest pain", "difficulty breathing"]): escal.append("Emergency")

    note = "**ESCALATE**:\n" + "\n".join(escal) if escal else "OK"
    note += "\n\n" + DISCLAIMER

    return symptoms_str, a_out, b_out, note

# **7.Final Nurse-Friendly UI**

In [13]:
import gradio as gr

with gr.Blocks(title="Nurse Screening Assistant") as demo:
    gr.Markdown("# 🩺 Medical Screening Assistant")
    gr.Markdown("**Educational prototype** • Model A: Logistic Regression • Model B: Flan-T5-base")

    gr.Markdown(
        f"<div style='color:red; font-weight:bold; font-size:16px; padding:10px; border:2px solid red;'>{DISCLAIMER}</div>"
    )

    inp = gr.Textbox(lines=5, label="Patient symptoms description")
    btn = gr.Button("Analyze", variant="primary")

    with gr.Row():
        with gr.Column():
            out_sym = gr.Textbox(label="Extracted Symptoms")
            out_a = gr.Textbox(lines=14, label="Model A - Logistic Regression (Top 5)")
        with gr.Column():
            out_b = gr.Textbox(lines=10, label="Model B - LLM Explanation")
            out_note = gr.Textbox(label="Safety / Escalation Note")

    btn.click(run_ui, inputs=inp, outputs=[out_sym, out_a, out_b, out_note])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.42.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://e4e8183e211c06abcc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e4e8183e211c06abcc.gradio.live
